<a href="https://colab.research.google.com/github/julopezrio/SYS/blob/main/Parcial1_SyS_2024_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1: Señales y Sistemas 2024-II

 ## Profesor: Andrés Marino Álvarez Meza, Ph.D.


## Departamento de Ingeniería Eléctrica, Electrónica, y Computación
## Universidad Nacional de Colombia - sede Manizales

# Instrucciones

-- Para recibir el crédito total, sus respuestas deben estar justificadas de manera clara, detallada y concreta, mostrando los procedimientos y razonamientos paso a paso.

-- Está permitido el uso de herramientas de inteligencia artificial (IA). Si las utiliza, por favor declare explícitamente cómo fueron empleadas en la resolución de cada pregunta. Incluya los prompts (consultas) y las iteraciones realizadas con las IA durante el desarrollo del parcial.

-- La entrega del parcial debe realizarse antes de las 23:59 del 5 de diciembre de 2024 al correo electrónico amalvarezme@unal.edu.co mediante un enlace de GitHub.

-- Los códigos deben estar debidamente comentados en las celdas correspondientes y explicados en celdas de texto (markdown). Los códigos que no incluyan comentarios ni discusiones no serán considerados en la evaluación final.

# Pregunta 1 (valor 2.5 puntos)

Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital de 5 bits con frecuencia de muestreo de $5kHz$, entrada análoga de -3.3 a 3.3 [v], aplicado a la señal continua $x(t) = 0.3 \cos(1000\pi t-\pi/4) +
0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t-\pi)$?. Realizar la simulación del proceso de digitalización incluyendo al menos 3 ciclos de la señal $x(t)$.

En caso de que la digitalización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada. El convesor debe permitir configurar la cantidad de bits, rango de la entrada análoga y la frecuencia de muestreo, indicándole al usuario si dicha frecuencia es apropiada o no, y graficar la señal continua, discreta y digital.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from scipy.spatial.distance import cdist
from scipy.signal import firwin, freqz, lfilter, filtfilt
import matplotlib
import scipy.signal as signal
%matplotlib inline
from ipywidgets import interact #crear graficos interactivos
from scipy import signal #crear señales prototipo en tiempo discreto



In [ ]:
#describir señal

n=100 # numero de muestras

#parametros funcion x1
w1=1000*np.pi
A1=0.3
f1=w1/(2*np.pi)
T1=1/f1
phi1=np.pi/4

#parametros funcion x2
w2=2000*np.pi
A2=0.6
f2=w2/(2*np.pi)
T2=1/f2

#parametros funcion x3
w3=11000*np.pi
A3=0.1
f3=w3/(2*np.pi)
T3=1/f3
phi3=np.pi

Fs=3*max(f1,f2,f3) # frecuecia de muestre que tiene que cumplir el teorema de nyquist
ts=1/Fs
ti=-5*min(T1,T2,T3) # tiempo inicial
tf=5*max(T1,T2,T3) # tiempo final

t = np.arange(ti,tf,ts)


#tb=np.linspace(ti,tf,k) # vector de tiempo

x1=A1*np.cos(w1*t-phi1)
x2=A2*np.sin(w2*t)
x3=A3*np.cos(w3*t-phi3)
x=x1+x2+x3


plt.plot(t,x)
plt.xlabel('tiempo [s]')
plt.ylabel('$A\sin[\omega t_k]$')
plt.show()

In [ ]:
def my_ceropen(x,ymin = -5, ymax = 5): #crea un arange de puntos en base a los valores maximos y minimos de y
  xmax = max(x) # x.max()
  xmin = min(x) # x.min()
  m = (ymax - ymin)/(xmax - xmin)
  c = ymin - m*xmin
  yv = m*x + c
  return yv

ycs = my_ceropen(x,ymin=-3.3,ymax=3.3) #uso de la funcion

#parametros de cuantizacion
nbits = 5 # cantidad de bits
rmin = -3.3
rmax = 3.3
ve = np.linspace(rmin,rmax,2**nbits)#crea los valores entre los intervalos defindos, y separandolos por espacios iguales que varian en base a la cantidad de bits

dn = cdist(ycs.reshape(-1,1),ve.reshape(-1,1)) #distancia entre todos los pares de puntos de dos arrays
dn.shape

ind=np.argmin(dn,axis=1)#argumento minimo
ind
print(ycs[1],ind[1],ve[ind[1]])

def my_cuantizador(yn, vq) : #yn punto a #cuantizar, vq vector de estados

  Ne = vq.shape[0] #tamaño vector de estados
  dn = cdist(yn.reshape(-1,1),vq.reshape(-1,1))#distancia yn a vector estados, reshape(-1,1) asegura vectores columna para poder utilizar cdist
  ind = np.argmin(dn) #posición distancia min
  return vq[ind]

for i in range(ycs.shape[0]):
   if i==0: #primera muestra
     yq = my_cuantizador(ycs[i],ve)
   else:
     yq = np.append(yq,my_cuantizador(ycs[i],ve))

plt.plot(t,ycs,c='y',label='ycs')#cs continua
plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.show()

plt.stem(t,ycs,label='ycs_dis')#cs discreta
plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.show()

#estados de cuantización

for v in ve:
  plt.plot(t,v*np.ones(len(t)),c='r')

# graficar señal Digital (discreta + cuantizada)

plt.plot(t,yq,c='r',label='ydig', linewidth=1)

plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.show()

# Pregunta 2 (valor 2.5 puntos)

Se dispone de un sistema modelado como una "caja negra" (ver celdas de código). Su tarea es analizar y comprobar mediante simulaciones si el sistema cumple con las propiedades de linealidad e invariancia en el tiempo. En caso de que el sistema sea lineal e invariante con el tiempo, determine su respuesta al impulso y utilice esta respuesta para calcular la salida del sistema ante la siguiente señal:

$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
# cargar sistema
FILEID = "1J9rhh0wWHZSBd8XmWGt1ZpCsMDuoUFmm"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O P1_model.zip && rm -rf /tmp/cookies.txt
!unzip -o P1_model.zip
!dir

In [ ]:
from P1_model import system_

#sistema pregunta 2
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs #frecuencia de muestreo
t = np.arange(-0.01, 5, 1/fs)  # Tiempo
signal_u = np.heaviside(t,1) # función heaviside #SEÑAL DE ENTRADA
#signal_u = np.heaviside(t,10) # función heaviside #SEÑAL DE ENTRADA
y_u = my_system.predict(signal_u)

# Visualización de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, signal_u, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_u, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

 Análisis de Linealidad
Para verificar la linealidad, necesitas comprobar si el sistema satisface:

$$
S({ax_1[n]}+bx_2[n])=aS(x_1)+bS(x_2)
$$
Para este caso debemos confirmar que si se aumenta nuestra señal de entrada, la señal de salida aumenta por el mismo factor

In [ ]:
#sistema pregunta 2
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs #frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo
signal_u = np.heaviside(t,1) # función heaviside-SEÑAL DE ENTRADA
y_u = my_system.predict(signal_u)# señal de salida

# Escalar la entrada por un factor a, el factor es arbitrario, es simplemente para ques cumpla homogeneidad
a = 2
u_scaled = a * signal_u #señal original por el factor el factor a escalar

# Salida del sistema para a*u(t)
y2 = my_system.predict(u_scaled)

# Escalar la salida original
y1_scaled = a * y_u

# Comparar gráficamente
plt.figure(figsize=(10, 6))
plt.stem(t, y2, label="Salida del sistema para a*u(t) (y2)")
plt.legend()
plt.title("Verificación de Linealidad: Homogeneidad")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")
plt.grid()
plt.show()
plt.figure(figsize=(10, 6))
plt.stem(t, y1_scaled, label="a * Salida original (y1_scaled)")
plt.legend()
plt.title("Verificación de Linealidad: Homogeneidad")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")
plt.grid()
plt.show()


Como se puede apreciar la amplitud tanto en la señal de entrada como en la de salida es igual por lo tanto el sistema es lineal, a continuacion se verifica si es invariante en el tiempo con un desplazamiento

In [ ]:
k=0.005 #valor del desplazamiento
# x[n - k]
x=np.heaviside(t-k,1) #señal de entrada desplazada
#salida del sistema para x[n - k]
y=my_system.predict(x) #señal de salida desplazada


# Comparar gráficamente
plt.figure(figsize=(12, 6))

# Gráfica de la salida del sistema para la entrada desplazada
plt.subplot(2, 1, 1)
plt.plot(t, x, label="entrada del sistema para x[n-k]")
plt.grid()
plt.legend()
plt.title("entrada del sistema para x[n-k]")

# Gráfica de la salida original desplazada manualmente
plt.subplot(2, 1, 2)
plt.plot(t, y, label="Salida desplazada y[n-k]", linestyle="--")
plt.legend()
plt.title("Salida original desplazada manualmente (y[n-k])")

plt.xlabel("Tiempo [s]")
plt.grid()
plt.tight_layout()
plt.show()


Como se nota en las graficas su escalorn incia en el mismo punto, por lo que la grafica es invariante en el tiempo, dado que lo es, el siguiente punto expresa hacer una convolucion de la salida de la siguiente señal
$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
g=np.diff(y_u) # funcion del pulso h[n]

#igualar las dimensiones de los array de g y t, para graficarlo
min_len = min(len(t), len(g))
t =t[:min_len]
x1=x1[:min_len]

#graficar en continua y discreta la respuesta al impulso
plt.plot(t,g,label='h[n]')
plt.legend()
plt.grid()
plt.xlabel('t')
plt.show()
plt.stem(t,g,label='h[n]')
plt.legend()
plt.grid()
plt.xlabel('t')
plt.show()

En las grafica anteriores se puede ver la respuesta al impulso de la señal de entrada, a continuacion se describen el vector de tiempo, el numero de muestras y la señal a calcular su salida en respuesta del impulso.

In [ ]:
tn=np.arange(-0.01,0.02,1/fs) #vector de tiempo
n=np.round(tn*fs) #numero de muestras en base a t=nTs

xn=np.sin( 100*np.pi*n)+np.sin(600*np.pi*n)#funcion x[n]

#grafica funcion x[n]
plt.figure(figsize=(12, 12))
plt.subplot(2, 1, 1)
plt.plot(n,xn,c='r',label='x[n]')
plt.stem(n,xn,label='x[n]')
plt.legend()
plt.grid()
plt.xlabel('t')
plt.show()


con=np.convolve(xn,g)

#redimensionar el array de la convolucion con el del numero de muestras
min_len = min(len(n), len(con))
n =n[:min_len]
con=con[:min_len]

#grafica de la convol
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(n,con,c='g',label='h[n]*x[n]')
plt.stem(n,con,label='h[n]*x[n]')
plt.legend()
plt.grid()
plt.xlabel('t')
plt.show()



en las graficas anteriores observamos el comportamiento de un tramo de la señal x[n] y luego la señal covolusionada con h[n].